In [1]:
import sys
import torch
import open3d as o3d
import os
import time
import numpy as np

sys.path.insert(0,"../")

from utils import lddmm_utils, mesh_processing, viz

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# torch type and device
use_cuda = torch.cuda.is_available()
torchdeviceId = torch.device("cuda:0") if use_cuda else "cpu"
torchdtype = torch.float32

# PyKeOps counterpart
KeOpsdeviceId = torchdeviceId.index  # id of Gpu device (in case Gpu is  used)
KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'

In [3]:
file_ref = "../data/preprocessed/bulldog_preprocessed.ply"
file_target = "../data/preprocessed/Labrador_preprocessed.ply"

mesh = o3d.io.read_triangle_mesh(file_ref)
VS, FS, RhoS = mesh_processing.getDataFromMesh(mesh)
VS, FS = torch.from_numpy(VS), torch.from_numpy(FS)

mesh = o3d.io.read_triangle_mesh(file_target)
VT, FT, RhoT = mesh_processing.getDataFromMesh(mesh)
VT, FT = torch.from_numpy(VT), torch.from_numpy(FT)

In [4]:
q0 = VS.clone().detach().to(dtype=torchdtype, device=torchdeviceId).requires_grad_(True)
VT = VT.clone().detach().to(dtype=torchdtype, device=torchdeviceId)
FS = FS.clone().detach().to(dtype=torch.long, device=torchdeviceId)
FT = FT.clone().detach().to(dtype=torch.long, device=torchdeviceId)
sigma = torch.tensor([10], dtype=torchdtype, device=torchdeviceId)

x, y, z = (
    q0[:, 0].detach().cpu().numpy(),
    q0[:, 1].detach().cpu().numpy(),
    q0[:, 2].detach().cpu().numpy(),
)
i, j, k = (
    FS[:, 0].detach().cpu().numpy(),
    FS[:, 1].detach().cpu().numpy(),
    FS[:, 2].detach().cpu().numpy(),
)

xt, yt, zt = (
    VT[:, 0].detach().cpu().numpy(),
    VT[:, 1].detach().cpu().numpy(),
    VT[:, 2].detach().cpu().numpy(),
)
it, jt, kt = (
    FT[:, 0].detach().cpu().numpy(),
    FT[:, 1].detach().cpu().numpy(),
    FT[:, 2].detach().cpu().numpy(),
)

In [5]:
save_folder, name = "doc/results/", "data.html"
os.makedirs(save_folder, exist_ok=True)

viz.show_meshes(VS, FS, VT, FT, save_folder, name, auto_open=False)

In [6]:
dataloss = lddmm_utils.lossVarifoldSurf(FS, VT, FT, lddmm_utils.GaussLinKernel(sigma=sigma))
Kv = lddmm_utils.GaussKernel(sigma=sigma)
loss = lddmm_utils.LDDMMloss(Kv, dataloss)

In [7]:
p0 = torch.zeros(q0.shape, dtype=torchdtype, device=torchdeviceId, requires_grad=True)

optimizer = torch.optim.LBFGS([p0], max_eval=10, max_iter=10)
print("performing optimization...")
start = time.time()

def closure():
    optimizer.zero_grad()
    L = loss(p0, q0)
    print("loss", L.detach().cpu().numpy())
    L.backward()
    return L

for i in range(2):
    print("it ", i, ": ", end="")
    optimizer.step(closure)

print("Optimization (L-BFGS) time: ", round(time.time() - start, 2), " seconds")

performing optimization...
it  0 : loss 73616610.0
loss 73573944.0
loss 192050910.0
loss 53562904.0
loss 51267670.0
loss 45881800.0
loss 38717550.0
loss 34596080.0
loss 31449136.0
loss 28581280.0
it  1 : loss 28581280.0
loss 26181600.0
loss 23384480.0
loss 22127104.0
loss 20725392.0
loss 19992736.0
loss 18951952.0
loss 18020176.0
loss 16865120.0
loss 16127616.0
Optimization (L-BFGS) time:  39.77  seconds


In [8]:
nt = 10
listpq = lddmm_utils.Shooting(p0, q0, Kv, nt=nt)

In [9]:
VTnp, FTnp = VT.detach().cpu().numpy(), FT.detach().cpu().numpy()
q0np, FSnp = q0.detach().cpu().numpy(), FS.detach().cpu().numpy()

viz.show_registration(VTnp, FTnp, FSnp, listpq, save_folder)

In [10]:
end = listpq[-1][1].detach().cpu().numpy()
V_approx = np.array([end[:,0], end[:,1], end[:,2]]).T
F_approx = np.array([FSnp[:,0], FSnp[:,1], FSnp[:,2]]).T

mesh_processing.export_mesh(V_approx, F_approx, "doc/results/objects/result.ply")

NameError: name 'np' is not defined